<a href="https://colab.research.google.com/github/erickuo5124/MLG_HW2/blob/main/LP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls '/content/drive/MyDrive/MLG/hw2/hw2_data'

dataset1  dataset2  dataset3  README.md


In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 3.8MB/s 
     |████████████████████████████████| 1.5MB 306kB/s 
     |████████████████████████████████| 194kB 7.4MB/s 
     |████████████████████████████████| 235kB 12.9MB/s 
     |████████████████████████████████| 2.2MB 14.6MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 


In [ ]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/MLG/hw2/hw2_data/dataset1/train.csv')
test = pd.read_csv('/content/drive/MyDrive/MLG/hw2/hw2_data/dataset1/test.csv')
content = pd.read_csv('/content/drive/MyDrive/MLG/hw2/hw2_data/dataset1/content.csv', delimiter='\t', header=None, index_col=0)
upload = pd.read_csv('/content/drive/MyDrive/MLG/hw2/hw2_data/dataset1/upload.csv')

In [ ]:
import torch
import networkx as nx
from torch_geometric.data import Data, ClusterData, ClusterLoader
from torch_geometric.utils import from_networkx

edge_index = torch.tensor([train['from'].values, train['from'].values], dtype=torch.long)
x = torch.tensor(content.sort_index().values, dtype=torch.float)
y = torch.tensor(train['label'].values, dtype=torch.float)
train_data = Data(edge_index=edge_index, x=x, y=y)

In [ ]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
  def __init__(self, hidden_layer):
    super(Net, self).__init__()
    torch.manual_seed(12345)
    self.conv1 = GCNConv(train_data.num_features, hidden_layer)
    self.conv2 = GCNConv(hidden_layer, hidden_layer)
    self.conv3 = GCNConv(hidden_layer, 2)
    self.classifier = Linear(2, 1)

  def forward(self, x, edge_index):
    h = self.conv1(x, edge_index)
    h = h.tanh()
    h = self.conv2(h, edge_index)
    h = h.tanh()
    h = self.conv3(h, edge_index)
    h = h.tanh()  # Final GNN embedding space.
    
    # Apply a final (linear) classifier.
    nodes = self.classifier(h)
    nodes = torch.squeeze(nodes)
    ids_from, ids_to = edge_index[0], edge_index[1]
    preds = torch.index_select(nodes, 0, ids_from) - torch.index_select(nodes, 0, ids_to)

    return torch.abs(preds)

In [ ]:
model = Net(128)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(100):
  optimizer.zero_grad()
  out = model(train_data.x, train_data.edge_index)
  loss = criterion(out, train_data.y)
  print(f'Epoch: {epoch:03d}, Loss: {loss:.6f}')
  loss.backward()
  optimizer.step()

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 000, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 001, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 002, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 003, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 004, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 005, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 006, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 007, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 008, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 009, Loss: 49.781258
tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SubBackward0>)
Epoch: 010, Loss: 49.781258